***Bird Species Observation Analysis***

In [1]:
import pandas as pd
import sqlalchemy
import plotly.express as px
import plotly.graph_objects as go

***Forest data collection***

In [2]:
#Reading Excelfile
forest_birds = pd.ExcelFile(r"E:\Mini_project_2\FOREST_Bird_Monitoring_Data_.XLSX")
#Storing Sheet names
sheet_names = forest_birds.sheet_names

sheets_dict = {sheet: forest_birds.parse(sheet) for sheet in sheet_names}


In [3]:
#Converting excell file into dataframe
forest_birds_df = pd.concat(
    [df.assign(Sheet=sheet_name) for sheet_name, df in sheets_dict.items()],
    ignore_index=True
)

In [ ]:
#Dropping sheets columns
forest_birds_df = forest_birds_df.drop(columns=['Sheet'])

In [ ]:
#Checking Duplicate values
print(forest_birds_df.duplicated().sum())

In [ ]:
#Checking null values
forest_birds_df.isna().sum()

In [7]:
#Dropping Sub_Unit_Code column
forest_birds_df = forest_birds_df.drop(columns=['Sub_Unit_Code'])


In [ ]:
forest_birds_df['ID_Method'].value_counts()

In [ ]:
forest_birds_df['ID_Method'].mode()

In [ ]:
#Filling null values with mode()
forest_birds_df['ID_Method'].fillna(forest_birds_df['ID_Method'].mode()[0],inplace=True)

In [ ]:
forest_birds_df['Distance'].value_counts()

In [ ]:
forest_birds_df['Distance'].mode()

In [ ]:
#Filling null values with mode()
forest_birds_df['Distance'].fillna(forest_birds_df['Distance'].mode()[0],inplace=True)

In [ ]:
forest_birds_df['Sex'].value_counts()

In [ ]:
forest_birds_df['Sex'].mode()

In [ ]:
#Filling null values with mode()
forest_birds_df['Sex'].fillna(forest_birds_df['Sex'].mode()[0],inplace=True)

In [ ]:
forest_birds_df.isna().sum()

In [ ]:
forest_birds_df['AcceptedTSN'].value_counts()

In [ ]:
print(forest_birds_df['AcceptedTSN'].median())


In [ ]:
#Filling null values with median()
forest_birds_df['AcceptedTSN'].fillna(forest_birds_df['AcceptedTSN'].median(),inplace=True)

In [ ]:
print(forest_birds_df.duplicated().sum())

In [ ]:
#Dropping Duplicate rows
forest_birds_final_df=forest_birds_df.drop_duplicates(keep='first') # it keeps fisrt rows

In [ ]:
forest_birds_final_df.describe()

In [ ]:
print(forest_birds_final_df.duplicated().sum())

***Grassland data collection***

In [27]:
#Reading Excelfile
gl_birds = pd.ExcelFile(r"E:\Mini_project_2\GRASSLAND_Bird_Monitoring_Data_.XLSX")
#Storing Sheet names
sheet_names = gl_birds.sheet_names

sheets_dict = {sheet: gl_birds.parse(sheet) for sheet in sheet_names}

In [ ]:
#Converting excell file into dataframe
gl_birds = pd.concat(
    [df.assign(Sheet=sheet_name) for sheet_name, df in sheets_dict.items()],
    ignore_index=True
)

In [ ]:
#Dropping sheets columns
gl_birds_df = gl_birds.drop(columns=['Sheet'])

In [ ]:
gl_birds_df=gl_birds_df.drop(columns=['Sub_Unit_Code'])

In [ ]:
print(gl_birds_df.duplicated().sum())

In [ ]:
gl_birds_df['ID_Method'].value_counts()

In [ ]:
gl_birds_df['ID_Method'].mode()


In [ ]:
gl_birds_df['ID_Method'].fillna(gl_birds_df['ID_Method'].mode()[0],inplace=True)


In [ ]:
gl_birds_df['Distance'].value_counts()      


In [ ]:
gl_birds_df['Distance'].mode()        


In [ ]:
gl_birds_df['Distance'].fillna(gl_birds_df['Distance'].mode()[0],inplace=True)

In [ ]:
gl_birds_df['AcceptedTSN'].value_counts()        

In [ ]:
print(gl_birds_df['AcceptedTSN'].median())     

In [ ]:
gl_birds_df['AcceptedTSN'].fillna(gl_birds_df['AcceptedTSN'].median(),inplace=True)          

In [ ]:
gl_birds_df['TaxonCode'].value_counts()  

In [ ]:
print(gl_birds_df['TaxonCode'].median() )  

In [ ]:
gl_birds_df['TaxonCode'].fillna(gl_birds_df['TaxonCode'].median(),inplace=True)  

In [ ]:
gl_birds_df.isna().sum()

In [ ]:
print(gl_birds_df.duplicated().sum())

In [ ]:
gl_birds_final_df=gl_birds_df.drop_duplicates(keep='first')

In [ ]:
print(gl_birds_final_df.duplicated().sum())

***Transfering data to SQL***

In [ ]:
#Forest monitoring data
from sqlalchemy import create_engine
host = "localhost"
port ="5432"
username = "postgres" 
password = "begin25"
db_name = "birds_data"

connector =f"postgresql://{username}:{password}@{host}:{port}/{db_name}" #must follow this syntax

engine = create_engine(connector)

table_name ="forest_birds_observation"

forest_birds_final_df.to_sql(table_name,engine,if_exists="replace",index= False)

print("Data Transferred Successfully")

In [ ]:
#grassland monitoring data
table_name ="grassland_birds_observation"

gl_birds_final_df.to_sql(table_name,engine,if_exists="replace",index= False)

print("Data Transferred Successfully")

**Temporal analysis**

*Seasonal Trends*

In [ ]:
#Seprating columns for analysis from main dataframe
seasons_forest=forest_birds_final_df[['Date','Common_Name']]

In [ ]:
#Seprating month from the date 
seasons_forest['Month']=seasons_forest['Date'].dt.month

In [ ]:
#Creating function to add season according to month
def get_season(month):
    if month in [12, 1, 2]: return 'Winter'
    elif month in [3, 4, 5]: return 'Spring'
    elif month in [6, 7, 8]: return 'Summer'
    else: return 'Fall'

seasons_forest['Season'] = seasons_forest['Month'].apply(get_season)

In [ ]:
SeasonalTrends_forest=seasons_forest.Common_Name.groupby(seasons_forest.Season).value_counts().reset_index(name="Sightings observed")

In [ ]:
#Seprating columns for analysis from main dataframe
seasons_grassland=gl_birds_final_df[['Date','Common_Name']]

In [ ]:
#Seprating month from the date
seasons_grassland['Month']=seasons_grassland['Date'].dt.month

In [ ]:
seasons_grassland['Season'] = seasons_grassland['Month'].apply(get_season)

In [ ]:
SeasonalTrends_grassland=seasons_grassland.Common_Name.groupby(seasons_grassland.Season).value_counts().reset_index(name="Sightings observed")

In [ ]:
#Forest Birds
st_forest=px.bar(SeasonalTrends_forest,x='Common_Name',y='Sightings observed',color='Season')
st_forest.update_layout(title="Seasonal trends in forest") 
st_forest.show()

In [ ]:
# Grassland birds
st_grassland=px.bar(SeasonalTrends_grassland,x='Common_Name',y='Sightings observed',color='Season')
st_grassland.update_layout(title="Seasonal trends in grassland") 
st_grassland.show()

*Observation Time*

In [ ]:
#Seprating columns for analysis from main dataframe
observation_forest=forest_birds_final_df[['Date', "Start_Time", "End_Time","Common_Name"]]

In [ ]:
#Changing datatypes of the columns
observation_forest['Date'] = observation_forest['Date'].astype(str)
observation_forest['Start_Time'] = observation_forest['Start_Time'].astype(str)


In [ ]:
#Merging two columns
observation_forest['Time']= observation_forest['Date'].astype(str) + ' ' + observation_forest['Start_Time']

In [ ]:
observation_forest['Time'] = pd.to_datetime(observation_forest['Time'])

# Extract hour
observation_forest['Time_of_observation'] = observation_forest['Time'].dt.hour


In [ ]:
observation_forest_activity = observation_forest['Time_of_observation'].value_counts().reset_index(name='Birds activity monitored')

In [ ]:
ot_forest=px.bar(observation_forest_activity,x='Time_of_observation',y='Birds activity monitored',color='Time_of_observation')
ot_forest.update_layout(title='Forest birds activity')
ot_forest.show()

In [ ]:
observation_grassland=gl_birds_final_df[['Date', "Start_Time", "End_Time","Common_Name"]]

In [ ]:
observation_grassland['Date'] = observation_grassland['Date'].astype(str)
observation_grassland['Start_Time'] = observation_grassland['Start_Time'].astype(str)

In [ ]:
observation_grassland['Time']= observation_grassland['Date'] + ' ' + observation_grassland['Start_Time']

In [ ]:
observation_grassland['Time'] = pd.to_datetime(observation_grassland['Time'])

# Extract hour
observation_grassland['Time_of_observation'] = observation_grassland['Time'].dt.hour

In [ ]:
observation_grassland_activity= observation_grassland['Time_of_observation'].value_counts().reset_index(name='Birds activity monitored')

In [ ]:
ot_grassland=px.bar(observation_grassland_activity,x='Time_of_observation',y='Birds activity monitored',color='Time_of_observation')
ot_grassland.update_layout(title='Grassland birds activity')
ot_grassland.show()

**Spatial Analysis**


*Location Insights*

In [ ]:
forest_loc=forest_birds_final_df[['Location_Type', 'Common_Name']]

In [ ]:
grassland_loc=gl_birds_final_df[['Location_Type', 'Common_Name']]

In [117]:
location_ins = pd.concat([forest_loc, grassland_loc], ignore_index=True)


In [ ]:
unique_species=location_ins.Common_Name.groupby(location_ins.Location_Type).nunique().reset_index(name="Species identified")


In [ ]:
location=px.bar(unique_species,x='Location_Type',y='Species identified',color='Location_Type')
location.update_layout(title=" Biodiversity hotspots") 
location.show()

**Plot-Level Analysis**

In [ ]:
plt_forest=forest_birds_final_df[['Plot_Name','Location_Type', 'Common_Name']]

In [ ]:
plt_analysis_forest=plt_forest.Common_Name.groupby(plt_forest.Plot_Name).nunique().reset_index(name='Species identified')

In [ ]:
plt_analysis_forest_ = plt_analysis_forest.sort_values(by="Species identified", ascending=False)

In [ ]:
plt_analysis_forest_10=plt_analysis_forest_.head(10)d

In [ ]:
pla_forest=px.bar(plt_analysis_forest_10,x='Plot_Name',y='Species identified',color='Plot_Name')
pla_forest.update_layout(title="Plots attract more species - Forest") 
pla_forest.show()

In [ ]:
plt_grassland=gl_birds_final_df[['Plot_Name','Location_Type','Common_Name']]

In [ ]:
plt_analysis_grassland=plt_grassland.Common_Name.groupby(plt_grassland.Plot_Name).nunique().reset_index(name='Species identified')

In [ ]:
plt_analysis_grassland_ = plt_analysis_grassland.sort_values(by="Species identified", ascending=False)
plt_analysis_grassland_10=plt_analysis_grassland_.head(10)

In [ ]:
pla_grassland=px.bar(plt_analysis_grassland_10,x='Plot_Name',y='Species identified',color='Plot_Name')
pla_grassland.update_layout(title="Plots attract more species - Grassland") 
pla_grassland.show()

*Species Analysis*

In [ ]:
forest_birds_species= forest_birds_final_df[['Location_Type', 'Scientific_Name']]

In [ ]:
gl_birds_species=gl_birds_final_df[['Location_Type', 'Scientific_Name']]

In [ ]:
birds_loc= pd.concat([forest_birds_species, gl_birds_species], ignore_index=True)

In [ ]:
diversity_count = birds_loc.groupby('Location_Type')['Scientific_Name'].nunique().reset_index(name='species found')

In [ ]:
dm=px.bar(diversity_count,x='Location_Type',y='species found',color='Location_Type')
dm.update_layout(title="Diversity Metric") 
dm.show()

Activity Patterns

In [145]:
ap_forest=forest_birds_final_df[['Interval_Length', 'ID_Method']]

In [146]:
ap_gl=gl_birds_final_df[['Interval_Length', 'ID_Method']]

In [ ]:
activity_forest = ap_forest['ID_Method'].value_counts().reset_index(name='observed_count')

In [ ]:
activity_grassland = ap_gl['ID_Method'].value_counts().reset_index(name='observed_count')

In [ ]:
forest_birds_acitivity=px.bar(activity_forest,x='ID_Method',y='observed_count',color='ID_Method')
forest_birds_acitivity.update_layout(title="Common activity in Forest") 
forest_birds_acitivity.show()

In [ ]:
grassland_birds_activity_=px.bar(activity_grassland,x='ID_Method',y='observed_count',color='ID_Method')
grassland_birds_activity_.update_layout(title="Common activity in Grassland ") 
grassland_birds_activity_.show()


**Environmental Conditions**

In [155]:
Weather_forest =forest_birds_final_df[['Temperature','Humidity','Sky', 'Wind', 'Disturbance','Distance']]

In [ ]:
Weather_forest_grouped= Weather_forest.groupby(['Temperature','Humidity', 'Sky','Wind'])['Distance'].value_counts().rename('Observed').reset_index()


In [ ]:
weather_distance_correlation_forest=px.scatter(Weather_forest_grouped,x='Temperature',y='Observed' ,hover_data=['Distance','Sky','Wind','Humidity'],color='Sky')
weather_distance_correlation_forest.update_layout(title='Forest weather analysis')
weather_distance_correlation_forest.show()

In [160]:
Weather_gl =gl_birds_final_df[['Temperature','Humidity','Sky', 'Wind', 'Disturbance','Distance']]

In [ ]:
grouped_gl= Weather_gl.groupby(['Temperature','Humidity', 'Sky','Wind'])['Distance'].value_counts().rename('Observed').reset_index()

In [ ]:
weather_distance_correlation_grassland=px.scatter(grouped_gl,x='Temperature',y='Observed' ,hover_data=['Distance','Sky','Wind','Humidity'],color='Sky')
weather_distance_correlation_grassland.update_layout(title='Grassland weather analysis')
weather_distance_correlation_grassland.show()

*Disturbance Effect*

In [ ]:
forest_de= Weather_forest.groupby(['Temperature','Humidity', 'Sky','Wind'])['Disturbance'].value_counts().rename('Observed counts').reset_index()

In [ ]:

dist_forest=px.scatter(forest_de,x='Temperature',y='Humidity' ,hover_data=['Disturbance','Sky','Wind','Observed counts'],color='Disturbance',size='Observed counts')
dist_forest.update_layout(title='Disturbance Effect in forest')
dist_forest.show()

In [ ]:
grassland_de= Weather_gl.groupby(['Temperature','Humidity', 'Sky','Wind'])['Disturbance'].value_counts().rename('Observed counts').reset_index()

In [ ]:
dist_grassland=px.scatter(grassland_de,x='Temperature',y='Humidity' ,hover_data=['Disturbance','Sky','Wind','Observed counts'],color='Disturbance',size="Observed counts")
dist_grassland.update_layout(title='Disturbance Effect in Grassland')
dist_grassland.show()

**Distance and Behavior**

*Distance Analysis*

In [180]:
ditance_data_forest=forest_birds_final_df[['Distance', 'Common_Name']]

In [ ]:
daf_count=ditance_data_forest.Common_Name.groupby(ditance_data_forest.Distance).size().reset_index(name='Observed counts')

In [ ]:
distance_analysis_forest=px.bar(daf_count,x='Distance',y='Observed counts',color='Distance')
distance_analysis_forest.update_layout(title="Distance Analysis of forest birds") 
distance_analysis_forest.show()

In [ ]:
daf_group=ditance_data_forest.Distance.groupby(ditance_data_forest.Common_Name).value_counts().reset_index(name="Observed counts")

In [ ]:

distance_analysis_species=px.bar(daf_group,x='Common_Name',y='Observed counts',color='Distance')
distance_analysis_species.update_layout(title="Distance Analysis of forest bird species") 
distance_analysis_species.show()

In [196]:
distance_data_gl=gl_birds_final_df[['Distance', 'Common_Name']]

In [ ]:
daf_gl=distance_data_gl.Common_Name.groupby(distance_data_gl.Distance).size().rename('observed count').reset_index()

In [ ]:
distance_analysis_gl=px.bar(daf_gl,x='Distance',y='observed count',color='Distance')
distance_analysis_gl.update_layout(title="Distance Analysis of grassland birds") 
distance_analysis_gl.show()

In [ ]:
daf_gl_species=distance_data_gl.Distance.groupby(distance_data_gl.Common_Name).value_counts().reset_index(name='Observed count')

In [ ]:
distance_analysis_gl_species=px.bar(daf_gl_species,x='Common_Name',y='Observed count',color='Distance')
distance_analysis_gl_species.update_layout(title="Distance Analysis of grassland bird species") 
distance_analysis_gl_species.show()

**Observer Trends**

*Observer Bias*

In [ ]:
observers_forest =forest_birds_final_df[['Observer','Scientific_Name']]

In [ ]:
observer_groupd_forest=observers_forest.Scientific_Name.groupby(observers_forest.Observer).nunique().reset_index(name='Total_observed_species')

In [ ]:
observer_count_forest=px.bar(observer_groupd_forest,x='Observer',y='Total_observed_species',color='Observer')
observer_count_forest.update_layout(title="Observers forest") 
observer_count_forest.show()

In [ ]:
observer_gl =gl_birds_final_df[['Observer','Scientific_Name']]

In [ ]:
observer_grouped_gl=observer_gl.Scientific_Name.groupby(observer_gl.Observer).nunique().reset_index(name='Total_observed_species')

In [ ]:
observer_count_gl=px.bar(observer_grouped_gl,x='Observer',y='Total_observed_species',color='Observer')
observer_count_gl.update_layout(title="Observers Grassland") 
observer_count_gl.show()

*Flyover Frequency*

In [219]:
Flyover_forest =forest_birds_final_df[['Common_Name', 'Flyover_Observed']]

In [ ]:
Flyover_forest_group=Flyover_forest.Flyover_Observed.groupby(Flyover_forest.Common_Name).value_counts().reset_index(name='observed_count')

In [ ]:
Flyover_freq_forest=px.bar(Flyover_forest_group,x='Common_Name',y='observed_count',color='Flyover_Observed')
Flyover_freq_forest.update_layout(title="Flyover_Observed forest") 
Flyover_freq_forest.show()

In [222]:
Flyover_gl=gl_birds_final_df[['Common_Name', 'Flyover_Observed']]

In [ ]:
Flyover_gl_group=Flyover_gl.Flyover_Observed.groupby(Flyover_gl.Common_Name).value_counts().reset_index(name='observed_count')

In [ ]:
Flyover_group_gl=px.bar(Flyover_gl_group,x='Common_Name',y='observed_count',color='Flyover_Observed')
Flyover_group_gl.update_layout(title="Flyover_Observed grassland") 
Flyover_group_gl.show()

*Visit Patterns*

In [225]:
forest_vp=forest_birds_final_df[['Common_Name', 'Visit']]

In [ ]:

# Count total bird observations per visit
visit_counts_forest = forest_vp.groupby('Visit').size().reset_index(name='Total_Observations')


In [ ]:
visit_counts_forest = px.bar(visit_counts_forest, x='Visit', y='Total_Observations', title="Bird Observations by Species",color="Visit")
visit_counts_forest.show()

In [ ]:
species_diversity_forest = forest_vp.groupby('Visit')['Common_Name'].nunique().reset_index(name='Unique_Species')

In [ ]:
species_diversity_visit = px.bar(species_diversity_forest, x='Visit', y='Unique_Species', title="Bird Observations by Species",color="Visit")
species_diversity_visit.show()

In [231]:
vp_gl=gl_birds_final_df[['Common_Name', 'Visit']]

In [ ]:
# Count total bird observations per visit
visit_counts_gl = vp_gl.groupby('Visit').size().reset_index(name='Total_Observations')


In [ ]:
visit_counts_gl = px.bar(visit_counts_gl, x='Visit', y='Total_Observations',  title="Bird Observations by Species",color="Visit")
visit_counts_gl.show()

In [ ]:
species_diversity_gl = vp_gl.groupby('Visit')['Common_Name'].nunique().reset_index(name='Unique_Species')

In [ ]:
species_diversity_gl = px.bar(species_diversity_gl, x='Visit', y='Unique_Species', title="Bird Observations by Species",color="Visit")
species_diversity_gl.show()

**Conservation Insights**

*Watchlist Trend*

In [ ]:
risk_forest=forest_birds_final_df[['Location_Type','Common_Name','Visit',  'PIF_Watchlist_Status', 'Regional_Stewardship_Status']]

In [ ]:
risk_forest_filter=risk_forest[(risk_forest['PIF_Watchlist_Status']==True) & (risk_forest['Regional_Stewardship_Status']==True)]

In [ ]:
risk_forest_filter_count=risk_forest_filter.value_counts(subset=['Common_Name',	'Visit'	,'PIF_Watchlist_Status','Regional_Stewardship_Status']).reset_index()
risk_forest_filter_count

In [ ]:
watchlist_forest = px.sunburst(risk_forest_filter_count,path=['Visit','Common_Name'],values='count',title="Forest birds watchlist")
watchlist_forest.show()

In [ ]:
risk_gl=gl_birds_final_df[['Location_Type','Common_Name','Visit',  'PIF_Watchlist_Status', 'Regional_Stewardship_Status']]

In [ ]:
risk_gl_filter=risk_gl[(risk_gl['PIF_Watchlist_Status']==True) & (risk_gl['Regional_Stewardship_Status']==True)]

In [ ]:
risk_gl_filter_count=risk_gl_filter.value_counts().reset_index()

In [ ]:
watchlist_gl = px.sunburst(risk_gl_filter_count,path=['Visit','Common_Name'],values='count',title="Grassland birds watchlist")
watchlist_gl.show()

*AOU Code Patterns*

In [ ]:
aou_forest=forest_birds_final_df[['Common_Name', 'AOU_Code','Regional_Stewardship_Status' ]]

In [ ]:
aou_forest_group=aou_forest.groupby('AOU_Code')[["Common_Name","Regional_Stewardship_Status"]].value_counts().reset_index()

In [ ]:
aou_forest_filter=aou_forest_group[aou_forest_group['Regional_Stewardship_Status']==True]

In [ ]:
forest_aou=px.bar(aou_forest_filter,x='AOU_Code',y='count',color='Common_Name')
forest_aou.update_layout(title="Forest conservation priorities") 
forest_aou.show()

In [ ]:
aou_gl=gl_birds_final_df[['Common_Name', 'AOU_Code','Regional_Stewardship_Status' ]]

In [ ]:
aou_count_gl=aou_gl.groupby('AOU_Code')[["Common_Name","Regional_Stewardship_Status"]].value_counts(ascending=True).reset_index()

In [ ]:
aou_gl_filter=aou_count_gl[aou_count_gl['Regional_Stewardship_Status']==True]

In [ ]:
grassland_aou=px.bar(aou_gl_filter,x='AOU_Code',y='count',color='Common_Name')
grassland_aou.update_layout(title="Grassland conservation priorities") 
grassland_aou.show()